In [1]:
from DatabaseConnections import *
import params
import pandas as pd
import cPickle as cpickle
import os
import csv
from IPython.display import clear_output
# clear_output()

STARTING...



In [2]:
db = PostgresDb(params.DB_HOST, params.DB_NAME) 

User name (RIS_TRAINING):shostetter
Password (RIS_TRAINING):········


In [3]:
@timeDec
def save_dictionaries(io='OUT'):
    """write out the data in dictionaries, may only be useful for testing..."""
    if io == 'OUT':
        cpickle.dump(params.nodeStreetNames, open(os.path.join(params.FOLDER, "nodeStreetNames.p"), "wb"))
        cpickle.dump(params.nodeIsIntersection, open(os.path.join(params.FOLDER, "nodeIsIntersection.p"), "wb"))
        cpickle.dump(params.nodeNextSteps, open(os.path.join(params.FOLDER, "nodeNextSteps.p"), "wb"))
        cpickle.dump(params.segmentBlocks, open(os.path.join(params.FOLDER, "segmentBlocks.p"), "wb"))
        cpickle.dump(params.node_master, open(os.path.join(params.FOLDER, "nodeMaster.p"), "wb"))
        cpickle.dump(params.clusterIntersections, open(os.path.join(params.FOLDER, "clusterIntersections.p"), "wb"))
        cpickle.dump(params.streetSet, open(os.path.join(params.FOLDER, "streetSet.p"), "wb"))
        cpickle.dump(params.mft1Dict, open(os.path.join(params.FOLDER, "mft1Dict.p"), "wb"))
        cpickle.dump(params.altGraph, open(os.path.join(params.FOLDER, "altGraph.p"), "wb"))
        cpickle.dump(params.mfts, open(os.path.join(params.FOLDER, "mfts.p"), "wb"))
        cpickle.dump(params.coordFromMaster, open(os.path.join(params.FOLDER, "coordFromMaster.p"), "wb"))
    else:
        d1 = cpickle.load(open(os.path.join(params.FOLDER, "nodeStreetNames.p"), "rb"))
        d2 = cpickle.load(open(os.path.join(params.FOLDER, "nodeIsIntersection.p"), "rb"))
        d3 = cpickle.load(open(os.path.join(params.FOLDER, "nodeNextSteps.p"), "rb"))
        d4 = cpickle.load(open(os.path.join(params.FOLDER, "segmentBlocks.p"), "rb"))
        d5 = cpickle.load(open(os.path.join(params.FOLDER, "nodeMaster.p"), "rb"))
        d6 = cpickle.load(open(os.path.join(params.FOLDER, "clusterIntersections.p"), "rb"))
        d7 = cpickle.load(open(os.path.join(params.FOLDER, "streetSet.p"), "rb"))
        d8 = cpickle.load(open(os.path.join(params.FOLDER, "mft1Dict.p"), "rb"))
        d9 = cpickle.load(open(os.path.join(params.FOLDER, "altGraph.p"), "rb"))
        d10 = cpickle.load(open(os.path.join(params.FOLDER, "mfts.p"), "rb"))
        d11 = cpickle.load(open(os.path.join(params.FOLDER, "coordFromMaster.p"), "rb"))
        return d1, d2, d3, d4, d5, d6, d7, d8, d9, d10, d11

In [4]:
def reset(db, version):
    tables = ['node_stnameft', 'lion', 'node', 'c_master_node',
              'c_master_segs', 'C_Lion_%s' % version, 'C_Lion_Nodes_%s' % version, 'm']
    for t in tables:
        print 'Droping %s\n' % t
        db.query("DROP TABLE if exists %s" % t)
    return raw_input('Re-imported lion and nodes? <enter>\n').upper()

In [5]:
# clean out old versions of lion ?

reset(db, params.VERSION)


Droping node_stnameft

Update sucessfull
Droping lion

Update sucessfull
Droping node

Update sucessfull
Droping c_master_node

Update sucessfull
Droping c_master_segs

Update sucessfull
Droping C_Lion_16d

Update sucessfull
Droping C_Lion_Nodes_16d

Update sucessfull
Droping m

Update sucessfull
Re-imported lion and nodes? <enter>



''

# Preprocess 
## Set up data schema

In [8]:
@timeDec
def pre_process(db, lion, node, version, highways=False):
    print 'Adding version number (%s)...\n' % version
    db.query("alter table %s add version varchar(50)" % lion)
    db.query("alter table %s add version varchar(50)" % node)
    db.query("update %s set version  = '%s'" % (lion, version))
    db.query("update %s set version  = '%s'" % (node, version))
    
    print 'Adding master ID fields to tables...\n'
    db.query("alter table %s add column mft int, add column masteridfrom int, add column masteridto int" % lion)
    db.query("alter table %s add column masterid int" % node)
    db.query("alter table %s add column is_int int" % node)
    
    print 'Creating node street name table (highways = %s)...\n' % str(highways)
    if highways:
        db.query("""drop table if exists node_stnameFT;
                    create table node_stnameFT as (
                    select node::int, street, 0 as master from (
                        select nodeidfrom as node, street
                        from {0}
                            where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C') and
                            (nonped = 'D' or nonped is null)
                            and street != 'UNNAMED STREET' and street not like '%{1}%'
                        group by nodeidfrom, street

                        union

                        select nodeidto as node, street
                        from {0}
                        where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C') and (nonped = 'D'
                        or nonped is null)
                        and street != 'UNNAMED STREET' and street not like '%{1}%'
                        group by nodeidto, street

                        union
                        /* ---------- ADD BACK IN PELHAM PKWY MAIN LINE ---------- */
                        select nodeidfrom as node, street
                        from {0}
                        where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C') and
                        (nonped = 'V' and street = 'PELHAM PARKWAY')
                        and street != 'UNNAMED STREET' and street not like '%{1}%'
                        group by nodeidfrom, street
                        /* ---------- ADD BACK IN PELHAM PKWY MAIN LINE ---------- */
                        union
                        /* ---------- ADD BACK IN FORDHAM RD MAIN LINE ---------- */
                        select nodeidfrom as node, street
                        from {0}
                        where nonped = 'V' and rb_layer in ('B', 'G') and street = 'EAST FORDHAM ROAD'
                        group by nodeidfrom, street
                        /* ---------- ADD BACK IN FORDHAM RD MAIN LINE ---------- */
                        ) ft
                    group by node, street)
                """.format(lion, 'PED OVPS'))
    else:
        db.query("""drop table if exists node_stnameFT;
                    create table node_stnameFT as (
                    select node::int, street, 0 as master
                    from (
                        select nodeidfrom as node, street
                        from {0}
                        where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C')
                        and street != 'UNNAMED STREET' and street not like '%{1}%'
                        group by nodeidfrom, street

                        union

                        select nodeidto as node, street
                        from {0}
                        where rb_layer in ('B', 'G') and featuretyp in ('0', '6', 'C')
                        and street != 'UNNAMED STREET' and street not like '%{1}%'
                        group by nodeidto, street
                        ) ft
                    group by node, street)
                """.format(lion, 'PED OVPS'))

        
# Run preprocess
pre_process(db, params.LION, params.NODE, params.VERSION, params.HIGHWAYS)
# Check resutls


Adding version number (16d)...

Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Adding master ID fields to tables...

Update sucessfull
Update sucessfull
Update sucessfull
Creating node street name table (highways = True)...

Update sucessfull
'pre_process' 59.67 sec


In [9]:
query_to_table(db, "select * from node_stnameFT limit 10")

,node,street,master
0,13,SHORE ROAD,0
1,14,PERTH AMBOY PLACE,0
2,14,WARDS POINT AVENUE,0
3,16,WARDS POINT AVENUE,0
4,17,SATTERLEE STREET,0
5,18,SATTERLEE STREET,0
6,18,SHORE ROAD,0
7,19,PERTH AMBOY PLACE,0
8,19,SATTERLEE STREET,0
9,21,AMBOY ROAD,0


# Part 2
### Build data dictionaries

In [10]:
def de_suffix(name):  # todo fix e as part of east example
    """cleans up street names for less percise matches"""
    d = {' WEST': ' DIR', ' EAST': ' DIR',
         ' SOUTH': ' DIR', ' NORTH': ' DIR',
         ' EXIT': ' DIR', ' ENTRANCE': ' DIR',
         ' APPROACH': ' DIR', ' NORTHBOUND': ' DIR',
         ' SOUTHBOUND': ' DIR', ' EASTBOUND': ' DIR',
         ' WESTBOUND': ' DIR'}  # new needs to be tested!!!!
    for i in d:
        if name not in ('WEST STREET', 'SOUTH STREET', 'NORTH STREET', 'EAST STREET',
                        'WEST AVENUE', 'SOUTH AVENUE', 'NORTH AVENUE', 'EAST AVENUE',
                        'WEST BOULEVARD', 'SOUTH BOULEVARD', 'NORTH BOULEVARD', 'EAST BOULEVARD',
                        'WEST LOOP', 'SOUTH LOOP', 'NORTH LOOP', 'EAST LOOP',
                        'WEST DRIVE', 'SOUTH DRIVE', 'NORTH DRIVE', 'EAST DRIVE',
                        'WEST ROAD', 'SOUTH ROAD', 'NORTH ROAD', 'EAST ROAD',
                        'JUNIPER BOULEVARD NORTH', 'JUNIPER BOULEVARD SOUTH'
                        'PROSPECT PARK WEST', 'AVENUE N', 'AVENUE S', 'AVENUE E',
                        'AVENUE W',):
            name = name.replace(i, d[i])
    return name 
    
@timeDec
def node_names(db, nodeStreetNames):
    data = db.query("select * from node_stnameFT")
    for row in data[0]:
        node, street = row[0], de_suffix(row[1])
        if node not in nodeStreetNames.keys():
            nodeStreetNames[node] = set([street])
        else:
            nodeStreetNames[node].add(street)
    return nodeStreetNames

params.nodeStreetNames = node_names(db, params.nodeStreetNames)

'node_names' 100.80 sec


In [11]:
@timeDec
def get_double_blocks(db, lion, node_table, highways=False):
    # get problem nodes where the blocks involved have more than 1 street name
    if highways:
        h = '''and ((nonped = 'D' or nonped is null or (nonped = 'V' and street = 'PELHAM PARKWAY'))
                or (nonped = 'V' and rb_layer in ('B', 'G')
                and street = 'EAST FORDHAM ROAD'))'''  # ADD BACK IN PELHAM PKWY
    else:
        h = ''
    db.query("""drop table if exists doubles; 
                create table doubles as (
                    select nf::int, nt::int, count(street)
                    from (
                        select  nodeidfrom as nf, nodeidto as nt, street
                        from {0}
                        where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G') {1}
                        and street != 'UNNAMED STREET' and street not like '%{2}%'
                        group by  nodeidfrom, nodeidto, street
                    union
                        select  nodeidto as nf, nodeidfrom as nt, street
                        from {0}
                        where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G') {1}
                        and street != 'UNNAMED STREET' and street not like '%{2}%'
                        group by  nodeidfrom, nodeidto, street
                    ) as t
                    group by nf, nt
                    having count(street) > 1)""".format(lion, h, 'PED OVPS'))
    
    # then those nodes need to have > 2 streets to be an intersection. 
    db.query("""create table temp_int as (
                    select nodeid, is_int, count(*) as cnt from (
                            select nodeid, is_int, street
                            from (
                                    select nodeid, is_int, street
                                    from {0} 
                                    join {1} on st_dwithin({0}.geom, {1}.geom,1)
                                    where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G') {2}
                                    and street != 'UNNAMED STREET' and street not like '%{3}%'
                                    group by nodeid, is_int, street
                            ) as base, 
                    doubles
                    where nodeid = nf or nodeid = nt
                            group by nodeid, is_int, street
                    ) as grp group by nodeid, is_int
                            having count(*) < 3
                );""".format(node_table, lion, h, 'PED OVPS'))
    # update revised nodes
    db.query("""update %s as n
                    set is_int = Null
                    from temp_int
                    where n.nodeid =temp_int.nodeid
                    and n.is_int = -1
                    ;""" % node_table)
    db.query("drop table temp_int; drop table doubles;")

@timeDec
def intersection_binary(db, node_table, lion, nodeIsIntersection, highways=False):
    """Updates the is_int field in db"""
    if highways:
        db.query("""create table temp_int as (
                    select nodeid, is_int, count(*) as cnt
                    from (
                        select nodeid, is_int, street
                        from {0} 
                        join {1} on st_dwithin({0}.geom, {1}.geom,1)
                        where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G') and 
                        (
                            (
                                nonped = 'D' or nonped is null or (nonped = 'V' and street = 'PELHAM PARKWAY')
                                or (nonped = 'V' and rb_layer in ('B', 'G') and street = 'EAST FORDHAM ROAD')
                            )
                        )--for highway exclution
                        and street != 'UNNAMED STREET' and street not like '%{2}%'
                        group by nodeid, is_int, street
                    ) as base
                    group by nodeid, is_int
                    having count(*) >1
                );""".format(node_table, lion, 'PED OVPS'))
    else:
        db.query("""create table temp_int as (
                    select nodeid, is_int, count(*) as cnt
                    from (
                        select nodeid, is_int, street
                        from {0} 
                        join {1} on st_dwithin({0}.geom, {1}.geom,1)
                        where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G')
                        and street != 'UNNAMED STREET' and street not like '%{2}%'
                        group by nodeid, is_int, street
                    ) as base
                    group by nodeid, is_int
                    having count(*) >1
                );""".format(node_table, lion, 'PED OVPS'))

    db.query("update {0} set is_int = -1 from temp_int where {0}.nodeid=temp_int.nodeid;".format(node_table))
    db.query("drop table temp_int;")


    # fix locations with double segments...
    get_double_blocks(db, lion, node_table, highways)  # 149.80 sec

    print 'DB updated'
    # get data from db into local dictionary
    data = db.query("select nodeid, is_int from %s " % node_table)  # where is_int =-1
    for i in data[0]:
        if i[1]:
            nodeIsIntersection[int(i[0])] = True
        else:
            nodeIsIntersection[int(i[0])] = False
    del data
    print 'Dictionary updated'
    return nodeIsIntersection

params.nodeIsIntersection = intersection_binary(db, params.NODE, params.LION, params.nodeIsIntersection, params.HIGHWAYS)

Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
Update sucessfull
'get_double_blocks' 22.97 sec
DB updated
Dictionary updated
'intersection_binary' 61.05 sec


In [12]:
def add_to_graph(street, node1, node2, nodeNextSteps):
    """checks if segment is in the graph adds it if not"""
    if node1 not in nodeNextSteps.keys():
        nodeNextSteps[node1] = {street: set([node2])}
    else:
        if street not in nodeNextSteps[node1].keys():
            nodeNextSteps[node1][street] = set([node2])
        else:
            nodeNextSteps[node1][street].add(node2)
    return nodeNextSteps


@timeDec
def graph(db, lion, nodeNextSteps, highways=False):
    """Graph the street network"""
    if highways:
        segs = db.query("""select street, segmentid, nodeidfrom, nodeidto 
                        from {0} 
                        where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G') and
                        ((nonped = 'D' or nonped is null or (nonped = 'V' and street = 'PELHAM PARKWAY'))
                        or (nonped = 'V' and rb_layer in ('B', 'G') and street = 'EAST FORDHAM ROAD'))
                        and street != 'UNNAMED STREET' and street not like '%{1}%'
                    """.format(lion, 'PED OVPS'))
    else:
        segs = db.query("""select street, segmentid, nodeidfrom, nodeidto 
                        from {0} 
                        where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G')
                        and street != 'UNNAMED STREET' and street not like '%{1}%'
                    """.format(lion, 'PED OVPS'))

    print 'Done with DB'
    for row in segs[0]:
        street, segmentid, nodeidfrom, nodeidto = row[0], row[1], int(row[2]), int(row[3])
        nodeNextSteps = add_to_graph(street, nodeidfrom, nodeidto, nodeNextSteps)
        nodeNextSteps = add_to_graph(street, nodeidto, nodeidfrom, nodeNextSteps)
    del segs
    return nodeNextSteps


params.nodeNextSteps = graph(db, params.LION, params.nodeNextSteps, params.HIGHWAYS)  # 179.22 sec

Done with DB
'graph' 198.38 sec


### Break point - save progress

In [13]:
save_dictionaries('OUT')  # 19.72 sec
# params.nodeStreetNames, params.nodeIsIntersection, params.nodeNextSteps, params.segmentBlocks, params.nodeMaster, params.clusterIntersections, params.streetSet, params.mft1Dict, params.altGraph, params.mfts, params.coordFromMaster = save_dictionaries('IN') #8.23 sec

'save_dictionaries' 2.45 sec


In [4]:
# run to read saved data back into memory 

params.nodeStreetNames, params.nodeIsIntersection, params.nodeNextSteps, params.segmentBlocks, params.nodeMaster, params.clusterIntersections, params.streetSet, params.mft1Dict, params.altGraph, params.mfts, params.coordFromMaster = save_dictionaries('IN') #8.23 se

'save_dictionaries' 1.95 sec


In [5]:
# simple graph search for grouping blocks

def go_to_end(street, done, todo, nodeIsIntersection=params.nodeIsIntersection, nodeNextSteps=params.nodeNextSteps):
    # anything left in the queue?
    if len(todo) == 0:
        # you're done exit
        return done
    else:
        # get new starting point
        start = todo.pop()
        # add it to the visited nodes
        done.append(start)
        # make sure it is not an ending point
        if not nodeIsIntersection[start]:
            # see where we can go from here
            for i in nodeNextSteps[start][street]:
                # make sure we haven't done it yet
                if i not in done:
                    # add it to the queue
                    todo.add(i)
        return go_to_end(street, done, todo)
    
@timeDec
def search(nodeStreetNames, streetSet, nodeIsIntersection, nodeNextSteps):
    """gets intersection to intersection segments or collection of segments"""
    for startNode in nodeStreetNames.keys():
        if nodeIsIntersection[startNode]:
            for street in nodeNextSteps[startNode].keys():
                queue = list(nodeNextSteps[startNode][street])
                while len(queue) > 0:
                    to_node = queue.pop()
                    streetSet.append(go_to_end(street, [startNode], set([to_node])))
    return nodeStreetNames, streetSet

params.nodeStreetNames, params.streetSet = search(params.nodeStreetNames, params.streetSet, params.nodeIsIntersection, params.nodeNextSteps)

print '\nnodeStreetNames(dict) sample: {{{0} : {1}}}'.format(params.nodeStreetNames.keys()[50], 
                                                             params.nodeStreetNames[params.nodeStreetNames.keys()[50]])

'search' 0.59 sec

nodeStreetNames(dict) sample: {52 : set(['CRAIG AVENUE', 'HYLAN BOULEVARD'])}


In [6]:
def write_csv(out_file, data_to_write, header=None):
    row_cnt = 0
    with open(out_file, 'wb') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')
        if header:
            writer.writerow(header)
        for row in data_to_write:
            writer.writerow(row)  # this writes the rows to the csv file row needs to be a list
            row_cnt += 1
    return str(row_cnt) + " rows were written to " + str(out_file)

In [7]:
@timeDec
def mft_round1(db, lion, altGraph, streetSet, mft1Dict, folder, highways=False):
    """gets simplest master segments based on from to nodes, intersections only"""
    if highways:
        sql = """select segmentid, nodeidfrom, nodeidto
                from {0} 
                where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G')
                and ((nonped = 'D' or nonped is null or (nonped = 'V' and street = 'PELHAM PARKWAY'))
                or (nonped = 'V' and rb_layer in ('B', 'G') and street = 'EAST FORDHAM ROAD'))
                and street != 'UNNAMED STREET' and street not like '%{1}%'
                group by segmentid, nodeidfrom, nodeidto""".format(lion, 'PED OVPS')
    else:
        sql = """select segmentid, nodeidfrom, nodeidto
                from {0}  
                where featuretyp in ('0', '6', 'C') and rb_layer in ('B','G')
                and street != 'UNNAMED STREET' and street not like '%{1}%'
                group by segmentid, nodeidfrom, nodeidto""".format(lion, 'PED OVPS')
    data = db.query(sql)
    # build local dict of seg node ids from-to to build mft
    for row in data[0]:
        # build up simple graph
        # node : [[other node , segid], [other node , segid]]
        segment, fromid, toid = row
        if int(fromid) not in altGraph.keys():
            altGraph[int(fromid)] = [[int(toid), segment]]
        else:
            altGraph[int(fromid)].append([int(toid), segment])

        if int(toid) not in altGraph.keys():
            altGraph[int(toid)] = [[int(fromid), segment]]
        else:
            altGraph[int(toid)].append([int(fromid), segment])
    print 'done graphing\n'
    mft = 0
    to_write_out = []
    for block in streetSet:  # streetSet is a set of nodes that together make up a block
        if block:
            print mft
            clear_output()
            mft += 1
            for node in block:
                for pair in altGraph[node]:
                    if pair[0] in block:  # both ends of the street are in the block
                        if mft not in mft1Dict.keys():
                            mft1Dict[mft] = []
                        mft1Dict[mft].append(pair[1])
                        to_write_out.append([mft, pair[1]])
    print 'Writing out csv\n'
    write_csv(os.path.join(folder, "mft.csv"), to_write_out, ['mft', 'segment'])  
    db.query('drop table if exists tempMaster; CREATE TABLE tempMaster (mft varchar(10), seg varchar(10))')
    return altGraph

params.altGraph = mft_round1(db,  params.LION, params.altGraph,  params.streetSet, params.mft1Dict, 
                             params.FOLDER, params.HIGHWAYS)  # 1367.67 sec

print '\naltGraph(dict) sample: {{{0} : {1}}}'.format(params.altGraph.keys()[50], 
                                                             params.nodeStreetNames[params.altGraph.keys()[50]])

Writing out csv

Update sucessfull
'mft_round1' 1328.75 sec

altGraph(dict) sample: {51 : set(['HYLAN BOULEVARD', 'SATTERLEE STREET'])}


In [8]:
raw_input("""PG Console \n\n \copy tempMaster FROM '%s\mft.csv' DELIMITERS ',' CSV HEADER;\n\nDone?""" % params.FOLDER)
clear_output()

In [10]:
@timeDec
def add_temp_table(db, lion):  # Assumes that tempMaster is imported
    db.query("""update %s as l
                    set mft = t.mft::int
                    from tempmaster as t
                    where l.segmentid =t.seg""" % lion)
    db.query("drop table tempmaster")

add_temp_table(db, params.LION)  # 35.44 sec

Update sucessfull
Update sucessfull
'add_temp_table' 135.45 sec


In [11]:
save_dictionaries('OUT')  # 10.07 sec

'save_dictionaries' 4.44 sec


In [12]:
# run to read saved data back into memory 

params.nodeStreetNames, params.nodeIsIntersection, params.nodeNextSteps, params.segmentBlocks, params.nodeMaster, params.clusterIntersections, params.streetSet, params.mft1Dict, params.altGraph, params.mfts, params.coordFromMaster = save_dictionaries('IN') #8.23 se

'save_dictionaries' 4.78 sec
